# Laboratorio: Métodos de búsqueda

En las clases anteriores creaste códigos para realizar búsquedas aleatorias (Simulated Annealing) y búsquedas dirigidas (Optimización Bayesiana). Estos métodos de búsqueda se utilizan para facilitar el proceso de optimización de funciones objetivos compleja y costosas de computar.

En este laboratorio usaremos el dataset de los diferentes tipos de iris, y sus longitudes y anchos de pétalos y sépalos. Utilizaremos un RandomForest para crear un modelo de clasificación y el métrico F1 para decidir cuál es el mejor modelo de acuerdo a lo que tenemos disponible.

1. Carga el dataset de Iris

In [63]:
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

2. Importa el archivo `Bosque.py`.

Este archivo contiene la función `RegresionBosque`, que recibe:
- X: las características independientes
- y: la variable de respuesta
- árboles: cantidad total de árboles
- profundidad de bosque: niveles de profundidad del bosque

Su salida es:
- modelo: El objeto con el modelo ajustado
- f1: El métrico que califica qué tan bueno es el modelo que se ajustó.


In [65]:
import Bosque
import numpy as np
import matplotlib.pyplot as plt

modelo, f1 = Bosque.RegresionBosque(X, y, 10, 3)

f1

0.9444444444444444

### Actividad 1:

Inicializa un espacio con 5 muestras en nuestro dominio de variables independientes:
- árboles: números enteros entre 5 y 50.
- profundidad: números enteros entre 2 y 10

Utiliza optimización Bayesiana para encontrar la combinación de árboles y profundidad que **maximice** el métrico F1.

In [80]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

arboles = np.random.randint(5, 51, 5).reshape(-1, 1)  
profundidad = np.random.randint(2, 11, 5).reshape(-1, 1)  

X_hyper = np.hstack((arboles, profundidad))  
valores_z = []


for i in range(len(arboles)):
    modelo, resultado = Bosque.RegresionBosque(X, y, int(arboles[i][0]), int(profundidad[i][0]))
    valores_z.append(resultado)

valores_z = np.array(valores_z)

n_iteraciones = 10
kernel = 1.0 * RBF(length_scale=1.0)

for i in range(n_iteraciones):
    
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(X_hyper, valores_z)
    
    x_tree = np.linspace(5, 50, 1000).reshape(-1, 1)
    x_profundidad = np.linspace(2, 10, 1000).reshape(-1, 1)
    x_hyper = np.hstack((x_tree, x_profundidad))
    
    y_pred, y_std = gp.predict(x_hyper, return_std=True)
    y_pred_upper = y_pred + 1.96 * y_std  
    
    i_next = np.argmax(y_pred_upper)
    siguiente_hyper = x_hyper[i_next]
    
    modelo, nuevo_valor = Bosque.RegresionBosque(X, y, int(siguiente_hyper[0]), int(siguiente_hyper[1]))
    X_hyper = np.vstack((X_hyper, siguiente_hyper))  s
    valores_z = np.hstack((valores_z, nuevo_valor))  


valores_z





array([0.95555556, 0.95555556, 0.93333333, 0.95555556, 0.95555556,
       0.93333333, 0.95555556, 0.95555556, 0.95555556, 0.95555556,
       0.94444444, 0.95555556, 0.94444444, 0.93333333, 0.93333333])

In [84]:
valores_z.max()

0.9555555555555556

### Actividad 2:

Inicializa 2 vectores con posibles valores para las variables independientes:
- árboles: números enteros entre 5 y 50
- profundidad: números enteros entre 2 y 10

Utiliza el algoritmo de Simulated Annealing que siga el siguiente orden:
- Elige un punto de partida para las variables.
- Selecciona al azar una de las dos para modificarlas.
- Selecciona un elemento al azar de la lista que contiene los posibles valores de esa variable.
- Sigue el algoritmo ($p$ y $q$) para decidir si usas esa combinación nueva o si mantienes la anterior.

In [95]:

valores_arboles = [i + 5 for i in range(46)]  
arreglo_arboles = np.array(valores_arboles).reshape([-1, 1])  
arreglo_profundidades = np.array([2, 3, 4, 5, 6, 7, 8, 9, 10]).reshape([-1, 1])  
temperatura = 1 

def generar_vector():
    arbol = arreglo_arboles[np.random.randint(len(arreglo_arboles))][0]
    profundidad = arreglo_profundidades[np.random.randint(len(arreglo_profundidades))][0]
    return np.array([arbol, profundidad])

vector_actual = generar_vector()


def actualizar_vector(vector_entrada):
    vector_temporal = vector_entrada.copy()
    indice_a_modificar = np.random.choice([0, 1])  

    if indice_a_modificar == 0:
        vector_temporal[0] = arreglo_arboles[np.random.randint(len(arreglo_arboles))][0]
    else:
        vector_temporal[1] = arreglo_profundidades[np.random.randint(len(arreglo_profundidades))][0]

    _, d1 = Bosque.RegresionBosque(X, y, vector_entrada[0], vector_entrada[1])
    _, d2 = Bosque.RegresionBosque(X, y, vector_temporal[0], vector_temporal[1])

    def calcular_probabilidad(delta1, delta2, temp):
        return np.exp((delta1 - delta2) / temp)

    q = calcular_probabilidad(d1, d2, temperatura)
    valor_random = np.random.rand() * 1.4

    if valor_random < q:
        return vector_temporal
    else:
        return vector_entrada

def aplicar_actualizaciones(vector, iteraciones):
    for _ in range(iteraciones):
        vector = actualizar_vector(vector)
    return vector

def recocido_simulado(temp, iteraciones):
    vector = generar_vector()
    while temp > 1:
        for _ in range(iteraciones):
            vector = aplicar_actualizaciones(vector, iteraciones)
        temp *= 0.9  
    return vector

def simulacion_final(temp, ciclos_max, iteraciones):
    resultados = []
    modelos = []
    for _ in range(ciclos_max):
        for _ in range(iteraciones):
            vector_final = recocido_simulado(temp, iteraciones)
            _, resultado = Bosque.RegresionBosque(X, y, vector_final[0], vector_final[1])
            modelos.append(vector_final)
            resultados.append(resultado)
    return resultados, modelos


ciclos_maximos = 4
iteraciones_por_ciclo = 4
resultados_finales, modelos_finales = simulacion_final(temperatura, ciclos_maximos, iteraciones_por_ciclo)



In [92]:
np.max(np.array(resultados_finales))

0.9555555555555556